In [ ]:
import os
import glob
from datetime import datetime
from multiprocessing import Pool

from matplotlib import gridspec
import matplotlib.pyplot as plt
from cmocean import cm

from cartopy.crs import LambertAzimuthalEqualArea
from cartopy.feature import LAND

from lmsiage.uncertainty import ComputeTotUncertainty

srs_dst = LambertAzimuthalEqualArea(central_longitude=0, central_latitude=90)
%matplotlib inline

In [ ]:
sid_dir = '../OSISAF_ice_drift_CDR_postproc'
sia_dir = '../NERSC_arctic25km_sea_ice_age_v2p1'
mesh_dir = f'{sia_dir}/mesh'
unc_dir = f'{sia_dir}/unc'
age_dir = f'{sia_dir}/age'
mesh_init_file = '../mesh_arctic_ease_25km_max7.npz'

In [ ]:
idate = datetime(1995, 12, 30)
oyear = '1995'
idate = datetime(2024, 12, 30)
oyear = '2024'

comp = ComputeTotUncertainty(mesh_dir, unc_dir, sid_dir, age_dir, mesh_init_file)
a, c, f, x, y, t = comp.load_age(idate)
unc_obs, len_f = comp.load_unc_obs(idate)
unc_myi = comp.load_unc_myi(idate)
unc_sid = comp.load_unc_sid(idate, unc_obs)
unc_sic_sid = comp.load_unc_sidsic(idate, unc_obs)
unc_frac = comp.compute_unc_frac(unc_obs, unc_myi, len_f)
unc_tot, unc_age = comp.compute_unc_tot(unc_frac, unc_sic_sid)


In [ ]:
fig = plt.figure(figsize=(14, 20))
gs = gridspec.GridSpec(7, 5, height_ratios=[20, 20, 20, 20, 20, 20, 3])

clims = [
    [0, 100],  # SIA fraction
    [0, 10],   # SIC Uncertainty
    [0, 300],  # Drift Uncertainty
    [0, 25],   # SID Uncertainty
    [0, 25]    # Total Uncertainty
]

cmaps = [
    cm.ice,      # SIA fraction
    'viridis',   # SIC Uncertainty
    'plasma',    # Drift Uncertainty
    'inferno',   # SID Uncertainty
    'magma'      # Total Uncertainty
]

cbar_labels = [
    'SIA fraction, %',
    'SIC Uncertainty, %',
    'Drift Uncertainty, km',
    'SID Uncertainty, %',
    'Total Uncertainty, %'
]

for row in range(6):
    arrays = [f[::-1][row], unc_frac[row], unc_sid[row], unc_sic_sid[row], unc_tot[row]]
    axs = []
    trps = []
    for col in range(5):
        ax = fig.add_subplot(gs[row, col], projection=srs_dst)
        trp = ax.tripcolor(x*1000, y*1000, t, arrays[col], clim=clims[col], cmap=cmaps[col])
        axs.append(ax)
        trps.append(trp)
    
    for ax in axs:
        ax.add_feature(LAND, facecolor='lightgray', edgecolor='black', zorder=1)
        ax.set_xlim([-1800000, 2000000])
        ax.set_ylim([-2300000, 2200000])
    axs[0].text(-1400000, -1400000, f'{row+1}YI', fontsize=14, zorder=2)

for col in range(5):
    ax_cbar = fig.add_subplot(gs[6, col])
    ax_cbar.set_visible(False)
    cbar0 = fig.colorbar(trps[col], ax=ax_cbar, orientation='horizontal', fraction=0.8, shrink=0.7)
    cbar0.set_label(cbar_labels[col])

plt.subplots_adjust(hspace=0, wspace=0.05)
plt.savefig(f'figure08_uncertainty_maps_{oyear}.png', bbox_inches='tight', dpi=150, pad_inches=0.1)
plt.show()

